In [3]:
import numpy as np
import scipy
import sys

np.set_printoptions(threshold=sys.maxsize)

Compute the fast laplacian for a 3D case for periodic boundary conditions on all boundaries, and check the results.

In [4]:
N = 10 # Number of points in a side of the domain.

# Trivial method

In [7]:
# Create the matrix for the 3D Laplacian operator.
# The unknown is stored as a vector with N^3 components, stored like in mif.
# The rhs has the same shape.
# Therefore, the matrix is a square N^3 x N^3 matrix.
# To compute the matrix, an easier representation as a 6D tensor is used.

A = np.zeros(shape=(N,N,N,N,N,N))

for i in range(N):
  for j in range(N):
    for k in range(N):
      # Derivative wrt x.
      A[i][j][k][i][j][k] += -2
      A[i][j][k][(i-1)%N][j][k] += 1
      A[i][j][k][(i+1)%N][j][k] += 1

      ## Derivative wrt y.
      A[i][j][k][i][j][k] += -2
      A[i][j][k][i][(j-1)%N][k] += 1
      A[i][j][k][i][(j+1)%N][k] += 1

      ## Derivative wrt z.
      A[i][j][k][i][j][k] += -2
      A[i][j][k][i][j][(k-1)%N] += 1
      A[i][j][k][i][j][(k+1)%N] += 1

A = A.reshape(N**3,N**3)

In [8]:
# Create a random solution.
np.random.seed(1)
xex = [np.random.random() for i in range(N**3)]
xex = np.array(xex)

In [9]:
# Compute the corresponding rhs.
b = A @ xex

In [10]:
# Check if the matrix construction is correct.
# Discretize u = sin(x)*sin(y)*sin(z) in [0, 2pi]^3
# and its laplacian rhs = -u.
u = np.empty(shape=(N,N,N))
rhs = np.empty(shape=(N,N,N))
h = 2*np.pi/N

for i in range(N):
  for j in range(N):
    for k in range(N):
      u[i][j][k] = np.sin(h*i) + np.sin(h*j) + np.sin(h*k)
      rhs[i][j][k] = -u[i][j][k]

u = u.reshape(N**3,)
rhs = rhs.reshape(N**3,)

# Check if the laplacian of u is rhs.
print(np.max((A/(h**2) @ u - rhs)))
# Correct convergence order (2).

# Check if the laplacian is the same if u is added a constant.
np.max((A @ (u+7) - A @ u))

0.09263896498548219


np.float64(7.993605777301127e-15)

# Intermediate method

In [35]:
# Compute eigenvector matrices.
Phi = np.empty(shape=(N,N), dtype=complex)

for i in range(N):
  for j in range(N):
    Phi[i][j] = np.exp(1j * 2 * np.pi * i * j / N) / np.sqrt(N)

H = Phi

In [36]:
# Compute btilde.
b_view = b.reshape(N,N,N)
btilde = np.empty(shape=(N, N, N), dtype=complex)

for i in range(N):
  for j in range(N):
    for k in range(N):
      total = 0
      for m in range(N):
        for n in range(N):
          for p in range(N):
            total += np.conj(Phi[m][i]) * np.conj(Phi[n][j]) * np.conj(Phi[p][k]) * b_view[m][n][p]
      btilde[i][j][k] = total

In [37]:
# Compute xtilde.
xtilde = np.copy(btilde)
xtilde[0,:,:] = 0
xtilde[:,0,:] = 0
xtilde[:,:,0] = 0
for i in range(1, N):
  for j in range(1, N):
    for k in range(1, N):
      xtilde[i][j][k] /= (2*np.cos(2 * np.pi * i / N)-2)
      xtilde[i][j][k] /= (2*np.cos(2 * np.pi * j / N)-2)
      xtilde[i][j][k] /= (2*np.cos(2 * np.pi * k / N)-2)

In [38]:
# Compute xtilde.
xtilde = np.copy(btilde)
xtilde[0,0,0] = 0
for i in range(N):
  for j in range(N):
    for k in range(N):
      eigx = (2*np.cos(2 * np.pi * i / N)-2)
      eigy = (2*np.cos(2 * np.pi * j / N)-2)
      eigz = (2*np.cos(2 * np.pi * k / N)-2)

      if not (i == 0 and j == 0 and k == 0):
        xtilde[i][j][k] /= (eigx + eigy + eigz)
      else:
        xtilde[i][j][k] = 0
xtilde2 = xtilde

In [39]:
# The result should be real. Verify this and remove the imaginary part.
tol = 1e-10
for i in range(N**3):
  assert abs(x[i].imag) < tol

xreal = np.empty(N**3)
for i in range(N**3):
  xreal[i] = x[i].real
x = xreal

In [40]:
# Check the result: xex and x should differ by a constant.
tol = 1e-10
constant = xex[i] - x[i]
for i in range(1,N):
  assert xex[i] - x[i] - constant < tol

# Fast method

In [11]:
# Compute btilde.
b_view = b.reshape(N,N,N)

# Step 1.
btilde1 = np.empty(shape=(N, N, N), dtype=complex)
for m in range(N):
  for n in range(N):
    btilde1[m,n,:] = scipy.fftpack.fft(b_view[m,n,:])

# Step 2.
btilde2 = np.copy(btilde1)
for k in range(N):
  for m in range(N):
    btilde2[k,:,m] = scipy.fftpack.fft(btilde1[k,:,m])

# Step 3.
btilde = np.copy(btilde2)
for j in range(N):
  for k in range(N):
    btilde[:,j,k] = scipy.fftpack.fft(btilde2[:,j,k])

In [12]:
# Compute xtilde.
xtilde = np.copy(btilde)
xtilde[0,0,0] = 0
for i in range(1,N):
  for j in range(N):
    for k in range(N):
      eigx = (2*np.cos(2 * np.pi * i / N)-2)
      eigy = (2*np.cos(2 * np.pi * j / N)-2)
      eigz = (2*np.cos(2 * np.pi * k / N)-2)
      xtilde[i][j][k] /= (eigx + eigy + eigz)
i = 0
for j in range (1, N):
    for k in range(N):
      eigx = (2*np.cos(2 * np.pi * i / N)-2)
      eigy = (2*np.cos(2 * np.pi * j / N)-2)
      eigz = (2*np.cos(2 * np.pi * k / N)-2)
      xtilde[i][j][k] /= (eigx + eigy + eigz)
i = 0
j = 0
for k in range(1, N):
      eigx = (2*np.cos(2 * np.pi * i / N)-2)
      eigy = (2*np.cos(2 * np.pi * j / N)-2)
      eigz = (2*np.cos(2 * np.pi * k / N)-2)
      xtilde[i][j][k] /= (eigx + eigy + eigz)
xtilde2 = xtilde

```
# Compute xtilde.
xtilde = np.copy(btilde)
xtilde[0,0,0] = 0
for i in range(N):
  for j in range(N):
    for k in range(N):
      eigx = (2*np.cos(2 * np.pi * i / N)-2)
      eigy = (2*np.cos(2 * np.pi * j / N)-2)
      eigz = (2*np.cos(2 * np.pi * k / N)-2)

      if not (i == 0 and j == 0 and k == 0):
        xtilde[i][j][k] /= (eigx + eigy + eigz)
      else:
        xtilde[i][j][k] = 0
xtilde2 = xtilde
```

In [13]:
# Compute x.
# Step 1.
x1 = np.empty(shape=(N, N, N), dtype=complex)
for m in range(N):
  for n in range(N):
    x1[...,m,n] = scipy.fftpack.ifft(xtilde[m,n,...])

# Step 2.
x2 = np.copy(x1)
for k in range(N):
  for m in range(N):
    x2[...,k,m] = scipy.fftpack.ifft(x1[k,m,...])

# Step 3.
x_view = np.copy(x2)
for j in range(N):
  for k in range(N):
    x_view[...,j,k] = scipy.fftpack.ifft(x2[j,k,...])

x = x_view.reshape((N**3,))

In [14]:
# The inverse fft returns a vector of complex numbers, but the imaginary part
# should be zero. Verify this and remove the imaginary part.
tol = 1e-10
for i in range(N**3):
  assert abs(x[i].imag) < tol

xreal = np.empty(N**3)
for i in range(N**3):
  xreal[i] = x[i].real
x = xreal

In [15]:
# Check the result: xex and x should differ by a constant.
tol = 1e-10
constant = xex[i] - x[i]
for i in range(1,N):
  assert xex[i] - x[i] - constant < tol

In [16]:
print(constant)

0.5006045994559047
